In [8]:
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
import numpy as np

In [62]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
housing = fetch_california_housing()

In [7]:
m,n = housing.data.shape

In [9]:
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [21]:
#正規方程式
x = tf.constant(housing_data_plus_bias, dtype=tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
xt = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xt,x)), xt), y)

In [18]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [20]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [58]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data= scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias =  np.c_[np.ones((m, 1)), scaled_housing_data.data]

In [59]:
housing.data.mean(), scaled_housing_data.mean(), housing.data.std(), scaled_housing_data.std()

(172.9556735813799,
 -1.1890574657535784e-15,
 621.6720986716564,
 0.9999999999999999)

In [60]:
# バッチ勾配降下
n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2 / m * tf.matmul(tf.transpose(x), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)
init = tf.global_variables_initializer()

In [61]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch: {}, MSE={}'.format(epoch, mse.eval()))
        sess.run(training_op)
    best_theta = theta.eval()
best_theta

Epoch: 0, MSE=13.791938781738281
Epoch: 100, MSE=0.7933831214904785
Epoch: 200, MSE=0.6139278411865234
Epoch: 300, MSE=0.5876743197441101
Epoch: 400, MSE=0.5703979730606079
Epoch: 500, MSE=0.5578980445861816
Epoch: 600, MSE=0.5488247871398926
Epoch: 700, MSE=0.5422334671020508
Epoch: 800, MSE=0.5374391078948975
Epoch: 900, MSE=0.5339478254318237


array([[ 2.0685523 ],
       [ 0.81975514],
       [ 0.1475465 ],
       [-0.18957382],
       [ 0.21768337],
       [ 0.00610024],
       [-0.04155179],
       [-0.6992431 ],
       [-0.6659777 ]], dtype=float32)

In [65]:
# gradient decent optimizer
# バッチ勾配降下
reset_graph()
n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [66]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [67]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch: {}, MSE={}'.format(epoch, mse.eval()))
        sess.run(training_op)
    best_theta = theta.eval()
best_theta

Epoch: 0, MSE=12.408011436462402
Epoch: 100, MSE=0.7551968693733215
Epoch: 200, MSE=0.5420873165130615
Epoch: 300, MSE=0.5331699252128601
Epoch: 400, MSE=0.5305383205413818
Epoch: 500, MSE=0.5287960767745972
Epoch: 600, MSE=0.5275489687919617
Epoch: 700, MSE=0.5266498327255249
Epoch: 800, MSE=0.5260008573532104
Epoch: 900, MSE=0.5255330801010132


array([[ 2.0685525e+00],
       [ 8.1063598e-01],
       [ 1.2685777e-01],
       [-2.0784083e-01],
       [ 2.4839847e-01],
       [-1.3083883e-03],
       [-3.9607048e-02],
       [-8.5861266e-01],
       [-8.2600272e-01]], dtype=float32)